In [1]:
import os
import pandas as pd
# Path to the dataset directory
dataset_path = '/kaggle/input/dataset9'

# List all files inside the dataset folder
files = os.listdir(dataset_path)
print("Files in dataset9:", files)

# Load the CSV file
df = pd.read_csv(f'{dataset_path}/final_mapped_categories.csv')

# Show first 5 rows
df.head()


import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)
import pandas as pd

df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')


Files in dataset9: ['final_mapped_categories.csv']
✅ Using device: cuda


In [3]:
# ✅ DISABLE wandb logging globally before importing Trainer
import os
os.environ["WANDB_DISABLED"] = "true"


✅ Using device: cuda


In [37]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(f'{dataset_path}/final_mapped_categories.csv')

# Show first 5 rows
df.head()


,Recipe URL,Recipe name,Preparation time,Servings,List of ingredients,List of instructions,Category,Region/Cuisine,Calories_Per_Serving,Broad_Category
0,https://www.food.com/recipe/cheesy-grits-casse...,cheesy grits casserole,65.0,6.0,4cupswater 1teaspoonsalt 1cupquickcooking grit...,heat water and salt to boiling gradually add g...,oven,Unknown,493.0,Bread
1,https://www.food.com/recipe/crock-pot-beer-bra...,crock pot beer brats,440.0,4.0,8bratwursts 2tablespoonsolive oil 2garlic clov...,heat oil and garlic in a large skillet add bra...,pork,Unknown,493.0,Meat/Poultry
2,https://www.food.com/recipe/banana-bread-with-...,banana bread with coconut milk,40.0,10.0,4ripe mashedbananas 712ounces canscoconut milk...,mix wet ingredients and dry ingredients mixtur...,quick breads,Unknown,493.0,Bread
3,https://www.food.com/recipe/pasta-salad-suprem...,pasta salad supreme,25.0,6.0,16ouncesrotini pastaor 16 ouncesshell pastasom...,cook pasta rinse under cold water and drain we...,vegetable,Unknown,493.0,Vegetable
4,https://www.food.com/recipe/bread-machine-monk...,bread machine monkey pull aparts,70.0,11.0,112teaspoonsactive dry yeast 114cupsbread flou...,add ingredients to your bread maker according ...,breads,Unknown,493.0,Bread


In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['Category'].fillna('Unknown'))


In [15]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device)  # 🚨 Important for GPU!
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [27]:
from tqdm import tqdm
import numpy as np

def get_bert_embeddings_batch(texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True,
                           truncation=True, max_length=128).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move back to CPU
        embeddings.extend(cls_embeddings)
    return np.array(embeddings)




In [28]:
bert_features = get_bert_embeddings_batch(df['text'].tolist(), batch_size=32)

 57%|█████▋    | 896/1563 [04:01<02:59,  3.71it/s]


KeyboardInterrupt: 

In [ ]:
import joblib
joblib.dump(bert_features, "bert_features.pkl")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# 🌟 Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✨ Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    bert_features, y, test_size=0.2, random_state=42
)

# 🌟 Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# 🔢 Determine number of output classes
num_classes = len(set(y))  # e.g., 2 for binary

# 🔧 Define model (Linear = Logistic Regression)
model = nn.Sequential(
    nn.Linear(X_train.shape[1], num_classes)  # 768 → classes
).to(device)

# 🔥 Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🧠 Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")


In [ ]:
# 🎯 Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_labels = torch.argmax(predictions, axis=1)

accuracy = (predicted_labels == y_test_tensor).float().mean()
print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")


In [ ]:
# Track loss over epochs
losses = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")
        
# Plotting
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Time")
plt.grid(True)
plt.show()


In [ ]:
import joblib
joblib.dump(le, "/kaggle/working/label_encoder.pkl")


In [ ]:
# 🔐 Save model weights
torch.save(model.state_dict(), "/kaggle/working/mlp_model.pt")
print("✅ Model saved successfully.")


In [ ]:
import numpy as np

if torch.is_tensor(bert_features):
    bert_features = bert_features.cpu().numpy()


In [ ]:
!pip install transformers datasets --quiet


In [30]:
import pickle
from sklearn.preprocessing import LabelEncoder

# 👇 Load the file
with open('/kaggle/working/label_encoder.pkl', 'rb') as f:
    obj = pickle.load(f)

# 🧠 Check what was saved
if isinstance(obj, LabelEncoder):
    le = obj
    print("✅ Loaded full LabelEncoder")
elif isinstance(obj, (list, tuple, np.ndarray)):
    le = LabelEncoder()
    le.classes_ = obj
    print("⚠️ Loaded class array, recreated LabelEncoder")
else:
    raise ValueError("❌ Unknown label_encoder.pkl format!")

# ✅ Get number of classes for model
num_labels = len(le.classes_)
print("Number of labels:", num_labels)


⚠️ Loaded class array, recreated LabelEncoder
Number of labels: 591


In [36]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42)
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)


In [37]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)


Map:   0%|          | 0/40009 [00:00<?, ? examples/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

In [38]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-39-9ab9ba8a0af5>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [ ]:
eval_results = trainer.evaluate()
print("📈 Evaluation Results:", eval_results)


In [ ]:
from sklearn.metrics import classification_report

preds = trainer.predict(test_ds)
pred_labels = preds.predictions.argmax(-1)

print(classification_report(test_df['label'], pred_labels, target_names=le.classes_))


In [ ]:
# 📦 Install required libraries
!pip install transformers datasets --quiet

# 🔽 Imports
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pickle



# 🧠 Combine text fields
df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')

# 🏷️ Encode target labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'].fillna('Unknown'))
num_labels = len(le.classes_)

# ✂️ Train-test split
train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42)

# 🔄 Convert to HuggingFace Dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# ✏️ Tokenize
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 🔧 Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# ⚙️ Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Replaces deprecated evaluation_strategy
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    disable_tqdm=False
)

# 🧠 Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)

# 🚀 Train the model
trainer.train()

# 📊 Evaluate
eval_results = trainer.evaluate()
print("📈 Evaluation:", eval_results)

# 💾 Save model and label encoder
model.save_pretrained("/kaggle/working/bert_finetuned/")
tokenizer.save_pretrained("/kaggle/working/bert_finetuned/")
with open("/kaggle/working/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/40009 [00:00<?, ? examples/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-d810f935d5a3>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

**2.**

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_scheduler
from tqdm import tqdm

# 📌 Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔥 Using device:", device)

# 📁 Load dataset
df = pd.read_csv('/kaggle/input/dataset9/final_mapped_categories.csv')
df['text'] = df['Recipe name'].fillna('') + ' ' + df['List of ingredients'].fillna('') + ' ' + df['List of instructions'].fillna('')

# 🔠 Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'].fillna('Unknown'))


# 🚫 Remove labels with <2 samples
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts > 1].index
df = df[df['label'].isin(valid_labels)]


# ✂️ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42, stratify=df['label']
)

# 🧙‍♂️ Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# 📦 Dataset class
class RecipeDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = RecipeDataset(train_encodings, train_labels)
val_dataset = RecipeDataset(val_encodings, val_labels)

# 📚 Dataloaders
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)

# 🧠 Load BERT with classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
model.to(device)

# ⚙️ Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_loader) * 4  # 4 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 🚀 Training loop
EPOCHS = 4
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print(f"📉 Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

    # ✅ Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    print(f"✅ Epoch {epoch+1} Validation Accuracy: {100 * correct / total:.2f}%")

print("🎉 Training complete! Krishna-Radha blessings on your BERT model 🦚✨")


🔥 Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

📉 Epoch 2 Loss: 1.9071
✅ Epoch 2 Validation Accuracy: 53.42%


Epoch 3 Training: 100%|██████████| 2491/2491 [15:22<00:00,  2.70it/s]


📉 Epoch 3 Loss: 1.6241
✅ Epoch 3 Validation Accuracy: 54.99%


Epoch 4 Training: 100%|██████████| 2491/2491 [15:22<00:00,  2.70it/s]


📉 Epoch 4 Loss: 1.4666
✅ Epoch 4 Validation Accuracy: 55.97%
🎉 Training complete! Krishna-Radha blessings on your BERT model 🦚✨


In [3]:
# 💾 Save the model
model.save_pretrained("bert_recipe_classifier")
tokenizer.save_pretrained("bert_recipe_classifier")


('bert_recipe_classifier/tokenizer_config.json',
 'bert_recipe_classifier/special_tokens_map.json',
 'bert_recipe_classifier/vocab.txt',
 'bert_recipe_classifier/added_tokens.json')

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification

# 🔁 Load your saved model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert_recipe_classifier")
model = BertForSequenceClassification.from_pretrained("bert_recipe_classifier")
model.to(device)  # ⚡ Send it to GPU again


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
EPOCHS = 9
start_epoch = 4  # Resume from here

for epoch in range(start_epoch, EPOCHS):
    model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        # ✅ Track training accuracy
        correct_train += (predictions == batch['labels']).sum().item()
        total_train += batch['labels'].size(0)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    train_acc = 100 * correct_train / total_train
    print(f"📉 Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")
    print(f"✅ Epoch {epoch+1} Training Accuracy: {train_acc:.2f}%")

    # 🧪 Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    val_acc = 100 * correct / total
    print(f"🧪 Epoch {epoch+1} Validation Accuracy: {val_acc:.2f}%\n")





Epoch 5 Training: 100%|██████████| 2491/2491 [14:21<00:00,  2.89it/s]


📉 Epoch 5 Loss: 1.4134
✅ Epoch 5 Training Accuracy: 62.81%
🧪 Epoch 5 Validation Accuracy: 55.97%



Epoch 6 Training: 100%|██████████| 2491/2491 [14:20<00:00,  2.90it/s]


📉 Epoch 6 Loss: 1.4105
✅ Epoch 6 Training Accuracy: 63.07%
🧪 Epoch 6 Validation Accuracy: 55.97%



Epoch 7 Training: 100%|██████████| 2491/2491 [14:19<00:00,  2.90it/s]


📉 Epoch 7 Loss: 1.4112
✅ Epoch 7 Training Accuracy: 62.78%
🧪 Epoch 7 Validation Accuracy: 55.97%



Epoch 8 Training: 100%|██████████| 2491/2491 [14:19<00:00,  2.90it/s]


📉 Epoch 8 Loss: 1.4132
✅ Epoch 8 Training Accuracy: 63.08%
🧪 Epoch 8 Validation Accuracy: 55.97%



Epoch 9 Training: 100%|██████████| 2491/2491 [14:20<00:00,  2.90it/s]


📉 Epoch 9 Loss: 1.4123
✅ Epoch 9 Training Accuracy: 63.09%
🧪 Epoch 9 Validation Accuracy: 55.97%



for 10 april

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔥 Using device:", device)

# ✅ Combine recipe name + ingredients + instructions
df['text'] = df['Recipe name'].fillna('') + ' ' + df['List of ingredients'].fillna('') + ' ' + df['List of instructions'].fillna('')

# ✅ Encode category
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'])

# ✅ Drop rare labels (less than 2 samples)
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts > 1].index
df = df[df['label'].isin(valid_labels)]

# ✅ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, stratify=df['label'], random_state=42
)

# ✅ Tokenizer — if RecipeBERT fails, fallback to BERT
try:
    tokenizer = AutoTokenizer.from_pretrained("recipegpt/recipe-bert")
    model_name = "recipegpt/recipe-bert"
except:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model_name = "bert-base-uncased"
    print("⚠️ RecipeBERT not found. Using BERT instead.")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256, return_tensors='pt')

# ✅ Dataset class
class RecipeDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = RecipeDataset(train_encodings, train_labels)
val_dataset = RecipeDataset(val_encodings, val_labels)

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_))
model.to(device)

# ✅ DataLoader
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)

# ✅ Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_loader) * 5
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# ✅ Training loop
EPOCHS = 5
for epoch in range(EPOCHS):
    print(f"\n🔁 Epoch {epoch+1}/{EPOCHS}")
    model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for batch in tqdm(train_loader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        correct_train += (predictions == batch['labels']).sum().item()
        total_train += batch['labels'].size(0)

    train_accuracy = 100 * correct_train / total_train
    print(f"📉 Train Loss: {total_loss/len(train_loader):.4f}")
    print(f"✅ Train Accuracy: {train_accuracy:.2f}%")

    # ✅ Validation
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
            predictions = torch.argmax(outputs.logits, dim=1)
            correct_val += (predictions == batch['labels']).sum().item()
            total_val += batch['labels'].size(0)

    val_accuracy = 100 * correct_val / total_val
    print(f"🧪 Val Loss: {val_loss/len(val_loader):.4f}")
    print(f"🧪 Val Accuracy: {val_accuracy:.2f}%")

print("\n🎉 Training complete!")


🔥 Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Training: 100%|██████████| 2491/2491 [30:38<00:00,  1.35it/s]


📉 Train Loss: 2.8594
✅ Train Accuracy: 38.03%


Validating: 100%|██████████| 623/623 [02:30<00:00,  4.14it/s]


🧪 Val Loss: 2.1106
🧪 Val Accuracy: 48.45%

🔁 Epoch 2/5


Training:  66%|██████▌   | 1646/2491 [20:22<10:27,  1.35it/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔥 Using device:", device)

# ✅ Combine recipe name + ingredients + instructions
df['text'] = df['Recipe name'].fillna('') + ' ' + df['List of ingredients'].fillna('') + ' ' + df['List of instructions'].fillna('')

# ✅ Encode category
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'])

# ✅ Drop rare labels (less than 2 samples)
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts > 1].index
df = df[df['label'].isin(valid_labels)]

# ✅ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, stratify=df['label'], random_state=42
)

# ✅ Tokenizer — if RecipeBERT fails, fallback to BERT
try:
    tokenizer = AutoTokenizer.from_pretrained("recipegpt/recipe-bert")
    model_name = "recipegpt/recipe-bert"
except:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model_name = "bert-base-uncased"
    print("⚠️ RecipeBERT not found. Using BERT instead.")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256, return_tensors='pt')

# ✅ Dataset class
class RecipeDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = RecipeDataset(train_encodings, train_labels)
val_dataset = RecipeDataset(val_encodings, val_labels)

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_))
model.to(device)

# ✅ DataLoader
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)

# ✅ Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_loader) * 5
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# ✅ Training loop
EPOCHS = 5
for epoch in range(EPOCHS):
    print(f"\n🔁 Epoch {epoch+1}/{EPOCHS}")
    model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for batch in tqdm(train_loader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        correct_train += (predictions == batch['labels']).sum().item()
        total_train += batch['labels'].size(0)

    train_accuracy = 100 * correct_train / total_train
    print(f"📉 Train Loss: {total_loss/len(train_loader):.4f}")
    print(f"✅ Train Accuracy: {train_accuracy:.2f}%")

    # ✅ Validation
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
            predictions = torch.argmax(outputs.logits, dim=1)
            correct_val += (predictions == batch['labels']).sum().item()
            total_val += batch['labels'].size(0)

    val_accuracy = 100 * correct_val / total_val
    print(f"🧪 Val Loss: {val_loss/len(val_loader):.4f}")
    print(f"🧪 Val Accuracy: {val_accuracy:.2f}%")

print("\n🎉 Training complete!")


🔥 Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Validating: 100%|██████████| 623/623 [02:20<00:00,  4.42it/s]


🧪 Val Loss: 1.6685
🧪 Val Accuracy: 55.92%

🔁 Epoch 4/5


Training: 100%|██████████| 2491/2491 [28:51<00:00,  1.44it/s]


📉 Train Loss: 1.4390
✅ Train Accuracy: 61.46%


Training:  14%|█▎        | 337/2491 [03:54<24:56,  1.44it/s]]

In [ ]:
c

🔥 Using device: cuda
⚠️ RecipeBERT not found. Using BERT instead.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



🔁 Epoch 1/10


Training: 100%|██████████| 2491/2491 [29:24<00:00,  1.41it/s]


📉 Train Loss: 2.8796
✅ Train Accuracy: 37.02%


...

Training: 100%|██████████| 2491/2491 [29:53<00:00,  1.39it/s]


📉 Train Loss: 1.2883
✅ Train Accuracy: 65.56%


Validating: 100%|██████████| 623/623 [02:26<00:00,  4.25it/s]


🧪 Val Loss: 1.6156
🧪 Val Accuracy: 57.15%

🔁 Epoch 7/10


Training:  96%|█████████▌| 2391/2491 [27:41<01:09,  1.45it/s]

In [7]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import DistilBertTokenizerFast, RobertaTokenizerFast

# Label encoding
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'])

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'label']])

# Load tokenizers
distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
roberta_tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Define tokenization functions
def tokenize_distilbert(example):
    return distilbert_tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

def tokenize_roberta(example):
    return roberta_tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

# Tokenize
tokenized_distilbert = dataset.map(tokenize_distilbert, batched=True)
tokenized_roberta = dataset.map(tokenize_roberta, batched=True)

# Split into train/test
tokenized_distilbert = tokenized_distilbert.train_test_split(test_size=0.2, seed=42)
tokenized_roberta = tokenized_roberta.train_test_split(test_size=0.2, seed=42)

# Remove unnecessary columns (for Trainer compatibility)
tokenized_distilbert = tokenized_distilbert.remove_columns(['text'])
tokenized_roberta = tokenized_roberta.remove_columns(['text'])

# Set format for PyTorch
tokenized_distilbert.set_format("torch")
tokenized_roberta.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/50012 [00:00<?, ? examples/s]

Map:   0%|          | 0/50012 [00:00<?, ? examples/s]

In [8]:

# 2. LOAD AND PREP DATA
# Assuming you've already loaded your data into `df` and cleaned it
# And you've created a combined text field already
df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')

# Label encoding
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'])

# Create Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'label']])

# 3. TOKENIZATION
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Remove text column and set format for PyTorch
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

# Train/test split
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# 4. LOAD ROBERTA MODEL
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(le.classes_)
)

# 5. TRAINING ARGUMENTS
training_args = TrainingArguments(
    output_dir="./roberta-results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_roberta",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

# 6. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

# 7. TRAINER
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

# 8. TRAIN
trainer.train()

# 9. EVALUATION
def evaluate_model(trainer, tokenized_dataset, model_name="Model"):
    print(f"\n📊 Evaluation for {model_name}")
    preds_output = trainer.predict(tokenized_dataset['test'])
    preds = np.argmax(preds_output.predictions, axis=1)
    labels = preds_output.label_ids

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    print("✅ Accuracy:", acc)
    print("✅ F1 Score:", f1)

    print("\n📝 Classification Report:")
    print(classification_report(labels, preds, target_names=le.classes_))

    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"{model_name} - Confusion Matrix")
    plt.show()

    return acc, f1

# Run evaluation
evaluate_model(trainer, tokenized_dataset, "RoBERTa")

# 10. LOSS PLOT
def plot_loss(trainer, model_name="Model"):
    logs = trainer.state.log_history
    train_loss, eval_loss, steps = [], [], []

    for log in logs:
        if 'loss' in log:
            train_loss.append(log['loss'])
            steps.append(log['step'])
        if 'eval_loss' in log:
            eval_loss.append(log['eval_loss'])

    plt.figure(figsize=(10, 5))
    plt.plot(steps[:len(train_loss)], train_loss, label="Training Loss")
    if eval_loss:
        plt.plot(steps[:len(eval_loss)], eval_loss, label="Validation Loss")
    plt.xlabel("Steps")
    plt.ylabel("Loss")
    plt.title(f"{model_name} - Loss Curve")
    plt.legend()
    plt.grid()
    plt.show()

# Plot loss
plot_loss(trainer, "RoBERTa")


Map:   0%|          | 0/50012 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [5]:
# 1. IMPORTS
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)

# 2. LOAD AND PREP DATA
# Reuse your existing `df` with combined text
df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')

# Label encoding
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'])

# Create Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'label']])

# 3. TOKENIZATION
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Remove text column and set format for PyTorch
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

# Train/test split
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# 4. LOAD DISTILBERT MODEL
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(le.classes_)
)

# 5. TRAINING ARGUMENTS
training_args = TrainingArguments(
    output_dir="./distilbert-results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_distilbert",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

# 6. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

# 7. TRAINER
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

# 8. TRAIN
trainer.train()

# 9. EVALUATION FUNCTION
def evaluate_model(trainer, tokenized_dataset, model_name="Model"):
    print(f"\n📊 Evaluation for {model_name}")
    preds_output = trainer.predict(tokenized_dataset['test'])
    preds = np.argmax(preds_output.predictions, axis=1)
    labels = preds_output.label_ids

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    print("✅ Accuracy:", acc)
    print("✅ F1 Score:", f1)

    print("\n📝 Classification Report:")
    print(classification_report(labels, preds, target_names=le.classes_))

    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"{model_name} - Confusion Matrix")
    plt.show()

    return acc, f1

# Run evaluation
evaluate_model(trainer, tokenized_dataset, "DistilBERT")

# 10. LOSS PLOT FUNCTION
def plot_loss(trainer, model_name="Model"):
    logs = trainer.state.log_history
    train_loss, eval_loss, steps = [], [], []

    for log in logs:
        if 'loss' in log:
            train_loss.append(log['loss'])
            steps.append(log['step'])
        if 'eval_loss' in log:
            eval_loss.append(log['eval_loss'])

    plt.figure(figsize=(10, 5))
    plt.plot(steps[:len(train_loss)], train_loss, label="Training Loss")
    if eval_loss:
        plt.plot(steps[:len(eval_loss)], eval_loss, label="Validation Loss")
    plt.xlabel("Steps")
    plt.ylabel("Loss")
    plt.title(f"{model_name} - Loss Curve")
    plt.legend()
    plt.grid()
    plt.show()

# Plot loss
plot_loss(trainer, "DistilBERT")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/50012 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [38]:
# 1. IMPORTS
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer
)



# Combine the text fields into a single column
df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')

# Encode the 'Category' column into numeric labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'])

# Confirm
df[['text', 'label']].head()


,text,label
0,cheesy grits casserole 4cupswater 1teaspoonsal...,407
1,crock pot beer brats 8bratwursts 2tablespoonso...,441
2,banana bread with coconut milk 4ripe mashedban...,450
3,pasta salad supreme 16ouncesrotini pastaor 16 ...,577
4,bread machine monkey pull aparts 112teaspoonsa...,51


In [39]:
from transformers import RobertaTokenizerFast

# Load RoBERTa tokenizer
roberta_tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# Create Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'label']])

# Tokenize the dataset
def tokenize(example):
    return roberta_tokenizer(example['text'], padding='max_length', truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Remove 'text' column (not needed for training) and set format for PyTorch
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

# Split into train/test sets
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Preview
tokenized_dataset


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/50012 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 40009
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 10003
    })
})

In [40]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer

# Load the RoBERTa model for classification
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(le.classes_)  # Number of unique recipe categories
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./roberta_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=9,
    weight_decay=0.01,
    logging_dir="./roberta_logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [41]:
from sklearn.metrics import accuracy_score, f1_score

# Define metric function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

# Start training 🚀
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.018300,1.947674,0.508347,0.449914
2,1.630500,1.707289,0.551635,0.492324
3,1.638700,1.625148,0.570629,0.529646


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=3753, training_loss=1.99568143311482, metrics={'train_runtime': 3424.8741, 'train_samples_per_second': 35.046, 'train_steps_per_second': 1.096, 'total_flos': 1.5873720190336512e+16, 'train_loss': 1.99568143311482, 'epoch': 3.0})

In [43]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=9,  # ⬅️ Increase to desired total epochs
    weight_decay=0.01,
    logging_dir="./roberta_logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [44]:
trainer.train(resume_from_checkpoint=True)



/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Epoch,Training Loss,Validation Loss,Accuracy,F1
4,1.463700,1.635222,0.563931,0.523022
5,1.330200,1.586402,0.570329,0.522752
6,1.243900,1.566359,0.571928,0.544195
7,1.072100,1.577766,0.577127,0.540842
8,1.097500,1.591536,0.582025,0.559292
9,0.950200,1.602031,0.583225,0.566046


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=11259, training_loss=0.8088135231408379, metrics={'train_runtime': 6824.25, 'train_samples_per_second': 52.765, 'train_steps_per_second': 1.65, 'total_flos': 4.762116057100954e+16, 'train_loss': 0.8088135231408379, 'epoch': 9.0})

In [9]:
# Install required libraries
!pip install -q transformers datasets scikit-learn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder

# Label Encoding
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Category'])
num_labels = len(label_encoder.classes_)
print("✅ Number of categories:", num_labels)

# Split dataset
train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42)

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load RecipeBERT
# model_name = "recipegpt/recipe-bert"
# model_name = "tae898/recipe-bert"
model_name = "alexdseo/RecipeBERT"  # Verified Hugging Face model



tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

# Tokenization function
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2
)

# Compute accuracy
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save model
trainer.save_model("/kaggle/working/recipebert-finetuned")
tokenizer.save_pretrained("/kaggle/working/recipebert-finetuned")

print("✅ Training complete. Model saved to /kaggle/working/recipebert-finetuned")


✅ Number of categories: 591


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-9-85a99c8524b8>:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.587700,1.711424,0.553534
2,1.584800,1.534349,0.579426
3,1.091700,1.494592,0.591722


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ Training complete. Model saved to /kaggle/working/recipebert-finetuned


**4**

In [10]:
# ✅ Install required libraries
!pip install -q transformers datasets scikit-learn

# ✅ Imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# ✅ Label Encoding
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Category'])
num_labels = len(label_encoder.classes_)
print("✅ Number of categories:", num_labels)

# ✅ Split dataset
train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42)

# ✅ Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# ✅ Load RecipeBERT model
model_name = "alexdseo/RecipeBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# ✅ Tokenization function
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

# ✅ Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# ✅ Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=9,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2
)

# ✅ Accuracy metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# ✅ Train the model
trainer.train()

# ✅ Save the model
trainer.save_model("/kaggle/working/recipebert-finetuned")
tokenizer.save_pretrained("/kaggle/working/recipebert-finetuned")
print("✅ Training complete. Model saved to /kaggle/working/recipebert-finetuned")

# ✅ Evaluate on test set
test_results = trainer.evaluate(test_dataset)
print(f"\n✅ Final Test Accuracy: {test_results['eval_accuracy']:.4f}")


✅ Number of categories: 591


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alexdseo/RecipeBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/40009 [00:00<?, ? examples/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-10-f2803cc449de>:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.560900,1.695374,0.555733
2,1.604000,1.555099,0.569229
3,1.101700,1.530210,0.585424
4,0.958500,1.526386,0.599820
5,0.868800,1.594048,0.593922
6,0.787700,1.710592,0.598620


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

✅ Training complete. Model saved to /kaggle/working/recipebert-finetuned


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



✅ Final Test Accuracy: 0.5998


In [9]:
# Final Kaggle-compatible, GPU-ready script with rare label removal, accuracy printing, and recipe classification setup

final_kaggle_ready_code = """
# ==================== RecipeBERT Fine-Tuning with Accuracy Output ===================='"""

!pip install -q transformers datasets scikit-learn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

# ✅ Combine structured text
df['text'] = (
    "Recipe: " + df['Recipe name'].fillna('') +
    ". Ingredients: " + df['List of ingredients'].fillna('') +
    ". Instructions: " + df['List of instructions'].fillna('')
)

# ✅ Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Category'])

# ✅ Drop classes with <2 samples to enable stratification
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts > 1].index
df = df[df['label'].isin(valid_labels)].reset_index(drop=True)

# ✅ Train/test split
train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42, stratify=df['label'])

# ✅ Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# ✅ Load RecipeBERT
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# ✅ Tokenization
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# ✅ Metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# ✅ Callback to print accuracy
class PrintMetricsCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        logs = state.log_history[-1]
        if "eval_accuracy" in logs:
            print(f"📈 Epoch {int(state.epoch)} - Validation Accuracy: {logs['eval_accuracy']:.4f}")

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    warmup_steps=100,
    fp16=True  # Enable mixed precision if on GPU
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1), PrintMetricsCallback()]
)

# ✅ Train
trainer.train()

# ✅ Save model
trainer.save_model("/kaggle/working/recipebert-finetuned-v2")
tokenizer.save_pretrained("/kaggle/working/recipebert-finetuned-v2")

# ✅ Final Evaluation
test_results = trainer.evaluate(test_dataset)
print(f"✅ Final Test Accuracy: {test_results['eval_accuracy']:.4f}")


# Save the corrected and complete training script
file_path = "/mnt/data/kaggle_recipebert_training_final.py"
with open(file_path, "w") as f:
    f.write(final_kaggle_ready_code)

file_path


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-9-c4559abf44b2>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,2.098300,1.892296,0.516414
2,1.690900,1.640451,0.553559
3,1.468300,1.531178,0.576247
4,1.344700,1.503520,0.581167
5,1.173100,1.501380,0.585383


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

✅ Final Test Accuracy: 0.5854


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/kaggle_recipebert_training_final.py'

**3****** "roberta-base"

In [10]:
# Final Kaggle-compatible, GPU-ready script with rare label removal, accuracy printing, and recipe classification setup

final_kaggle_ready_code = """
# ==================== RecipeBERT Fine-Tuning with Accuracy Output ===================='"""

!pip install -q transformers datasets scikit-learn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

# ✅ Combine structured text
df['text'] = (
    "Recipe: " + df['Recipe name'].fillna('') +
    ". Ingredients: " + df['List of ingredients'].fillna('') +
    ". Instructions: " + df['List of instructions'].fillna('')
)

# ✅ Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Category'])

# ✅ Drop classes with <2 samples to enable stratification
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts > 1].index
df = df[df['label'].isin(valid_labels)].reset_index(drop=True)

# ✅ Train/test split
train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42, stratify=df['label'])

# ✅ Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# ✅ Load RecipeBERT
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# ✅ Tokenization
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# ✅ Metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# ✅ Callback to print accuracy
class PrintMetricsCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        logs = state.log_history[-1]
        if "eval_accuracy" in logs:
            print(f"📈 Epoch {int(state.epoch)} - Validation Accuracy: {logs['eval_accuracy']:.4f}")

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    warmup_steps=100,
    fp16=True  # Enable mixed precision if on GPU
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1), PrintMetricsCallback()]
)

# ✅ Train
trainer.train()

# ✅ Save model
trainer.save_model("/kaggle/working/recipebert-finetuned-v2")
tokenizer.save_pretrained("/kaggle/working/recipebert-finetuned-v2")

# ✅ Final Evaluation
test_results = trainer.evaluate(test_dataset)
print(f"✅ Final Test Accuracy: {test_results['eval_accuracy']:.4f}")


# Save the corrected and complete training script
file_path = "/mnt/data/kaggle_recipebert_training_final.py"
with open(file_path, "w") as f:
    f.write(final_kaggle_ready_code)

file_path


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/39844 [00:00<?, ? examples/s]

Map:   0%|          | 0/9961 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-10-12b64bc53cb8>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,2.122000,1.887083,0.512800
2,1.704200,1.652469,0.547234
3,1.493900,1.531836,0.568116
4,1.369700,1.502980,0.575946
5,1.203300,1.497248,0.576147
6,1.097400,1.504450,0.585885
7,0.986700,1.522902,0.590804
8,0.877500,1.553591,0.590704


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

✅ Final Test Accuracy: 0.5908


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/kaggle_recipebert_training_final.py'

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"
